# Дополнительные оценки качества моделей

## Импорт необходимых библиотек

In [1]:
import os
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
from transformers import pipeline
from scipy.sparse import hstack
from datasets import Dataset
from copy import deepcopy
import regex
import re
import emoji

## Чтение тестовых данных

### Данные с чистыми сообщениями

`data/test_clear.json` – `.json` файл структуры `[{"text": "blablabla", "label": 0}]`, где ключ `text` представляет текстовое значение сообщения, а ключ `label` – класс сообщения (0: безопасное, 1: опасное).  
Сообщения были собраны из чата "СТАНКИН" смежного Telegram канала.

In [2]:
df_clear = pd.read_json('../data/test_clear.json')

In [3]:
df_clear_short = df_clear.head(1000)

In [4]:
df_clear_short.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1000 non-null   object
 1   label   1000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


In [5]:
df_clear_short.head()

,text,label
0,Очень похоже)❤️,0
1,Грозный СТАНКИН))),0
2,,0
3,"Здравствуйте, а куда можно написать по вопроса...",0
4,"Здравствуйте, напишите свой вопрос в данный ча...",0


### Данные со спам сообщениями

`data/parsed_lols_2023.txt` – `.txt` файл с набором спам-сообщений.  
Файл был получен с [lols.bot](https://lols.bot/2023/)

In [6]:
parsed_lols_2023 = open('../data/parsed_lols_2023.txt', 'r', encoding='utf-8').readlines()

In [7]:
for i in range(len(parsed_lols_2023)):
    parsed_lols_2023[i] = parsed_lols_2023[i].replace('<br/>', '', 1).strip().replace('<br/>', '\n')

In [8]:
len(parsed_lols_2023)

32437

In [9]:
parsed_lols_2023_short = parsed_lols_2023[:999]

In [10]:
df_spam = pd.DataFrame()
df_spam['text'] = parsed_lols_2023_short
df_spam['label'] = [1] * len(parsed_lols_2023_short)

In [11]:
df_spam.head()

,text,label
0,🤝ƃАЗA ЧAT𝖮B ᎩCЛᎩГ🤝 + Б𝖮Т AВTОРАСCЫЛKИ🚀<br>\n❗️...,1
1,دانیالللل,1
2,🤙🏻🤘🏻Caмo𝚎 cлoжное нa сеpoм рынке — это найти н...,1
3,𝙇 𝘼 𝙈 𝙄 𝘼,1
4,🔲DROP SERVICE🔲<br><br>☕️ 𝔸ℝ𝔸𝔹𝕀ℂ𝔸×ℝ𝕆𝔹𝕌𝕊𝕋𝔸 ☕️ \n...,1


### Объединение данных

In [12]:
df_test = pd.concat([df_clear_short, df_spam])

In [13]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1999 entries, 0 to 998
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1999 non-null   object
 1   label   1999 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 46.9+ KB


In [14]:
df_test.head()

,text,label
0,Очень похоже)❤️,0
1,Грозный СТАНКИН))),0
2,,0
3,"Здравствуйте, а куда можно написать по вопроса...",0
4,"Здравствуйте, напишите свой вопрос в данный ча...",0


## Обработка данных

### Создание новых столбцов с числовыми данными

В спам сообщениях очень часто используются эмодзи, ссылки, теги (`@`), много новых строк (`\n`) и лишних пробелов, так что их количество в сообщении может играть большую роль в классификации сообщений

#### Функции для подготовки текста и дополнительных числовых данных

> Эти же функции находятся в `../utils/preprocessing.py`.

In [15]:
def count_emojis(text: str) -> int:
    """
    Считает количество эмодзи в тексте.

    Параметры:
        text (str): Исходный текст.

    Возвращает:
        int: Количество найденных эмодзи.
    """
    # Разбиваем текст на "графемные кластеры" (учитывая составные эмодзи)
    data = regex.findall(r'\X', text)
    # Считаем эмодзи
    emoji_counter = sum(emoji.is_emoji(word) for word in data)
    return emoji_counter

In [16]:
def count_newlines(text: str) -> int:
    """
    Считает количество символов новой строки (\n) в тексте.

    Параметры:
        text (str): Исходный текст.

    Возвращает:
        int: Количество символов новой строки.
    """
    # Подсчёт символов новой строки
    newline_count = text.count('\n')
    return newline_count

In [17]:
def count_whitespaces(text: str) -> int:
    """
    Считает количество последовательных пробелов (два или более) в тексте.

    Параметры:
        text (str): Исходный текст.

    Возвращает:
        int: Количество найденных последовательных пробелов.
    """
    # Считаем последовательности из двух и более пробелов
    extra_spaces_count = len(re.findall(r'\s{2,}', text))
    return extra_spaces_count

In [18]:
# Регулярное выражение для ссылок
url_pattern = r'(https?://[^\s]+|www\.[^\s]+|[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}(?:/[^\s]*)?)'

In [19]:
def count_links(text: str) -> int:
    """
    Считает количество ссылок в тексте.

    Параметры:
        text (str): Исходный текст.

    Возвращает:
        int: Количество найденных ссылок.
    """
    # Найти все ссылки в тексте
    links = re.findall(url_pattern, text)
    return len(links)

In [20]:
def count_tags(text: str) -> int:
    """
    Считает количество упоминаний (тегов) в тексте.

    Параметры:
        text (str): Исходный текст.

    Возвращает:
        int: Количество найденных тегов.
    """
    # Подсчёт символов "@" как маркеров тегов
    tag_count = text.count('@')
    return tag_count

Удаление пунктуации и эмодзи, приведение к нижнему регистру может снизить шум и сложность модели

In [21]:
def remove_emojis(text: str) -> str:
    """
    Удаляет все эмодзи из текста.

    Параметры:
        text (str): Исходный текст.

    Возвращает:
        str: Текст без эмодзи.
    """
    # Разбиваем текст на "графемные кластеры"
    data = regex.findall(r'\X', text)
    # Удаляем все эмодзи
    text_without_emojis = ''.join(word for word in data if not emoji.is_emoji(word))
    return text_without_emojis

Ссылки и тэги без пунктуации превращаются в набор латинских символов, что будет мешать работе модели, но при этом их нельзя удалять из текстов, так как это важные маркеры. Поэтому их необходимо заменить на что-то вроде `[LINK]` и `[TAG]`

In [22]:
def replace_links(text: str) -> str:
    """
    Заменяет все ссылки в тексте на "[LINK]".

    Параметры:
        text (str): Исходный текст.

    Возвращает:
        str: Текст с заменёнными ссылками.
    """
    # Заменить ссылки на [LINK]
    text = re.sub(url_pattern, '[LINK]', text)
    return text

In [23]:
def replace_tags(text: str) -> str:
    """
    Заменяет все упоминания (теги) в тексте на "[TAG]".

    Параметры:
        text (str): Исходный текст.

    Возвращает:
        str: Текст с заменёнными тегами.
    """
    # Регулярное выражение для тегов
    tag_pattern = r'@[a-zA-Z0-9_]+'
    # Замена тегов на [TAG]
    text = re.sub(tag_pattern, '[TAG]', text)
    return text

In [24]:
def preprocess_text(
        text: str,
        lower: bool = True,
        remove_punctuation: bool = True,
        remove_emoji: bool = True,
        remove_whitespaces: bool = True,
        remove_links: bool = True,
        remove_tags: bool = True
) -> str:
    """
    Предобрабатывает текст, включая нормализацию регистра, удаление пунктуации, эмодзи, ссылок, тегов и пробелов.

    Параметры:
        text (str): Исходный текст.
        lower (bool): Приводить ли текст к нижнему регистру (по умолчанию True).
        remove_punctuation (bool): Удалять ли пунктуацию (по умолчанию True).
        remove_emoji (bool): Удалять ли эмодзи (по умолчанию True).
        remove_whitespaces (bool): Удалять ли лишние пробелы (по умолчанию True).
        remove_links (bool): Удалять ли ссылки (по умолчанию True).
        remove_tags (bool): Удалять ли теги (по умолчанию True).

    Возвращает:
        str: Обработанный текст.
    """
    # Приведение текста к нижнему регистру
    if lower:
        text = text.lower()
    # Замена ссылок на [LINK]
    if remove_links:
        text = replace_links(text)
    # Замена тегов на [TAG]
    if remove_tags:
        text = replace_tags(text)
    # Удаление пунктуации
    if remove_punctuation:
        text = re.sub(r'[^\w\s]', '', text)  # Удаляет пунктуацию
        # Восстановление [LINK] после очистки пунктуации
        if remove_links:
            text = text.replace('LINK', '[LINK]')
        # Восстановление [TAG] после очистки пунктуации
        if remove_tags:
            text = text.replace('TAG', '[TAG]')
    # Удаление эмодзи
    if remove_emoji:
        text = remove_emojis(text)
    # Удаление лишних пробелов и символов новой строки
    if remove_whitespaces:
        text = re.sub(r'\s+', ' ', text).strip()
    return text

#### Подготовка текста и дополнительных числовых данных

In [25]:
emojis_count = []
newlines_count = []
whitespaces_count = []
links_count = []
tags_count = []
for text in df_test['text']:
    emojis_count.append(count_emojis(text))
    newlines_count.append(count_newlines(text))
    whitespaces_count.append(count_whitespaces(text))
    links_count.append(count_links(text))
    tags_count.append(count_tags(text))

In [26]:
df_test['emojis'] = emojis_count
df_test['newlines'] = newlines_count
df_test['whitespaces'] = whitespaces_count
df_test['links'] = links_count
df_test['tags'] = tags_count

In [27]:
df_test.head()

,text,label,emojis,newlines,whitespaces,links,tags
0,Очень похоже)❤️,0,1,0,0,0,0
1,Грозный СТАНКИН))),0,0,0,0,0,0
2,,0,0,0,0,0,0
3,"Здравствуйте, а куда можно написать по вопроса...",0,1,0,0,0,0
4,"Здравствуйте, напишите свой вопрос в данный ча...",0,0,2,1,1,0


In [28]:
texts = list(deepcopy(df_test['text']))
for text_i in range(len(texts)):
    temp = preprocess_text(texts[text_i])
    if not temp:
        temp = np.NaN
    texts[text_i] = temp

In [29]:
texts

['очень похоже',
 'грозный станкин',
 nan,
 'здравствуйте а куда можно написать по вопросам общежития туда попадают на конкурсной основе какие примерно шансы',
 'здравствуйте напишите свой вопрос в данный чат [LINK]',
 'день рождения станкина',
 nan,
 nan,
 'подскажите всем ли дают общежитие',
 'а что делать тем кто был не допущен ждать 07072023 или сразу идти в военкомат ребёнок из многодетной семьи',
 'сегодня был сказали ждать до 07072023 а чего ждать если отчисляют или ещё чтото то пусть сразу говорят ребёнку тогда нужно подавать заявку в другой или в армию идти почему преподаватели молчали ни кто с родителями не связывался даже в колледже такого безобразия нет',
 'это очень круто предлагаем пообщаться по теме проекта более плотно как найти юлию и елизавету девочки поставьте на это сообщение если вы готовы обсудить возможности приземления вашего проекта в станкине или напишите на почту prostankin[LINK] пожалуйста',
 'как закалялся станкиновец написал бы николай островский',
 'альбе

In [30]:
len(texts)

1999

In [31]:
df_test['text_preprocessed'] = texts

In [32]:
df_test.head()

,text,label,emojis,newlines,whitespaces,links,tags,text_preprocessed
0,Очень похоже)❤️,0,1,0,0,0,0,очень похоже
1,Грозный СТАНКИН))),0,0,0,0,0,0,грозный станкин
2,,0,0,0,0,0,0,NaN
3,"Здравствуйте, а куда можно написать по вопроса...",0,1,0,0,0,0,здравствуйте а куда можно написать по вопросам...
4,"Здравствуйте, напишите свой вопрос в данный ча...",0,0,2,1,1,0,здравствуйте напишите свой вопрос в данный чат...


In [33]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1999 entries, 0 to 998
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   text               1999 non-null   object
 1   label              1999 non-null   int64 
 2   emojis             1999 non-null   int64 
 3   newlines           1999 non-null   int64 
 4   whitespaces        1999 non-null   int64 
 5   links              1999 non-null   int64 
 6   tags               1999 non-null   int64 
 7   text_preprocessed  1892 non-null   object
dtypes: int64(6), object(2)
memory usage: 140.6+ KB


### Удаление лишних строк

In [34]:
sum(df_test.isnull().sum(axis = 1))

107

In [35]:
df_test.dropna(subset=['text_preprocessed'], inplace=True)

In [36]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1892 entries, 0 to 998
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   text               1892 non-null   object
 1   label              1892 non-null   int64 
 2   emojis             1892 non-null   int64 
 3   newlines           1892 non-null   int64 
 4   whitespaces        1892 non-null   int64 
 5   links              1892 non-null   int64 
 6   tags               1892 non-null   int64 
 7   text_preprocessed  1892 non-null   object
dtypes: int64(6), object(2)
memory usage: 133.0+ KB


In [37]:
df_test.head()

,text,label,emojis,newlines,whitespaces,links,tags,text_preprocessed
0,Очень похоже)❤️,0,1,0,0,0,0,очень похоже
1,Грозный СТАНКИН))),0,0,0,0,0,0,грозный станкин
3,"Здравствуйте, а куда можно написать по вопроса...",0,1,0,0,0,0,здравствуйте а куда можно написать по вопросам...
4,"Здравствуйте, напишите свой вопрос в данный ча...",0,0,2,1,1,0,здравствуйте напишите свой вопрос в данный чат...
5,День рождения СТАНКИНА 🎉,0,1,0,0,0,0,день рождения станкина


In [38]:
count_zero = df_test[df_test['label'] == 0].shape[0]

In [39]:
count_zero

893

In [40]:
count_one = df_test[df_test['label'] == 1].shape[0]

In [41]:
count_one

999

Итого мы получили 1892 строк, из которых:
- 893 чистых сообщений
- 999 сообщений, помеченных как спам

## Загрузка моделей

In [42]:
models_path = "../models"  # Путь к папке с моделями

Загрузка scaler и vectorizer

In [43]:
with open(os.path.join(models_path, "scaler.pkl"), "rb") as scaler_file:
    scaler = pickle.load(scaler_file)

with open(os.path.join(models_path, "vectorizer.pkl"), "rb") as vectorizer_file:
    vectorizer = pickle.load(vectorizer_file)

Загрузка классических моделей

In [44]:
classic_models = {}
for file in os.listdir(models_path):
    if file.endswith(".pkl") and file not in ["scaler.pkl", "vectorizer.pkl"]:
        with open(os.path.join(models_path, file), "rb") as model_file:
            classic_models[file] = pickle.load(model_file)

Загрузка моделей rubert

In [45]:
rubert_models = {
    "rubert_tiny2": {
        "path": os.path.join(models_path, "finetuned_rubert_tiny2"),
    },
    "rubert_tiny2_p": {
        "path": os.path.join(models_path, "finetuned_rubert_tiny2_p"),
    },
}
for name, model_info in rubert_models.items():
    model_info["classifier"] = pipeline("text-classification", model=model_info["path"], truncation=True)

Device set to use cuda:0
Device set to use cuda:0


## Предсказание значений

### Функции для предсказания значений

Функция предсказания для классических моделей

In [46]:
def predict_message(text: str, vectorizer, scaler, model) -> tuple[int, list[float]]:
    """
    Предсказывает метку и вероятности для текста с использованием модели машинного обучения.

    Параметры:
        text (str): - входной текст для анализа
        vectorizer: - векторизатор для преобразования текста
        scaler: - масштабатор числовых признаков
        model: - модель машинного обучения

    Возвращает:
        tuple[int, list[float]] - метка предсказания и вероятности
    """
    # Предварительная обработка текста
    text_preprocessed = preprocess_text(text)

    # Векторизация текста
    text_vector = vectorizer.transform([text_preprocessed])

    # Создание числовых признаков
    numerical_features = pd.DataFrame(
        [[count_emojis(text), count_newlines(text), count_whitespaces(text), count_links(text), count_tags(text)]],
        columns=['emojis', 'newlines', 'whitespaces', 'links', 'tags']
    )
    # Масштабирование числовых признаков
    numerical_features_scaled = scaler.transform(numerical_features)

    # Объединение текстовых и числовых признаков
    features = hstack([text_vector, numerical_features_scaled])

    # Предсказание
    prediction = model.predict(features)
    probabilities = model.predict_proba(features)
    return int(prediction[0]), *probabilities.tolist()

Функция предсказания для моделей rubert

In [47]:
def bert_predict_batch(messages: list[str], classifier, threshold: float = 0.5) -> list[tuple[int, list[float]]]:
    """
    Функция для батчевого предсказания с помощью модели BERT через pipeline classifier.

    Параметры:
        messages (list[str]): – список сообщений для анализа
        classifier (pipeline): – pipeline классификатор
        threshold (float): – пороговое значение для предсказания класса

    Возвращает:
        list[tuple[int, list[float]]] - список предсказаний (0 или 1) и вероятностей
    """
    results = classifier(messages, batch_size=8)
    predictions = []
    for result in results:
        prediction = int(result['label'][-1])  # Извлечение метки предсказания
        score = result['score']  # Вероятность предсказания
        probabilities = [1 - score, score] if prediction == 1 else [score, 1 - score]

        if probabilities[1] < threshold:
            prediction = 0

        predictions.append((prediction, probabilities))

    return predictions

### Сбор предсказаний

In [48]:
# Подготовка данных для BERT моделей
dataset = Dataset.from_pandas(df_test)

for name, model_info in rubert_models.items():
    rubert_results = bert_predict_batch(dataset["text"], model_info["classifier"], threshold=0.945)
    df_test[name] = [pred[0] for pred in rubert_results]

In [49]:
df_test.head()

,text,label,emojis,newlines,whitespaces,links,tags,text_preprocessed,rubert_tiny2,rubert_tiny2_p
0,Очень похоже)❤️,0,1,0,0,0,0,очень похоже,0,0
1,Грозный СТАНКИН))),0,0,0,0,0,0,грозный станкин,0,0
3,"Здравствуйте, а куда можно написать по вопроса...",0,1,0,0,0,0,здравствуйте а куда можно написать по вопросам...,0,0
4,"Здравствуйте, напишите свой вопрос в данный ча...",0,0,2,1,1,0,здравствуйте напишите свой вопрос в данный чат...,1,1
5,День рождения СТАНКИНА 🎉,0,1,0,0,0,0,день рождения станкина,0,1


### Сравнение

In [50]:
for name, model_info in rubert_models.items():
    print(name, '\n')
    print('Classification report:\n', classification_report(df_test['label'], df_test[name]))
    print("Confusion Matrix:\n", confusion_matrix(df_test['label'], df_test[name]))
    print('\n\n')

rubert_tiny2 

Classification report:
               precision    recall  f1-score   support

           0       0.94      0.98      0.96       893
           1       0.98      0.94      0.96       999

    accuracy                           0.96      1892
   macro avg       0.96      0.96      0.96      1892
weighted avg       0.96      0.96      0.96      1892

Confusion Matrix:
 [[877  16]
 [ 60 939]]



rubert_tiny2_p 

Classification report:
               precision    recall  f1-score   support

           0       0.99      0.91      0.95       893
           1       0.93      0.99      0.96       999

    accuracy                           0.96      1892
   macro avg       0.96      0.95      0.96      1892
weighted avg       0.96      0.96      0.96      1892

Confusion Matrix:
 [[817  76]
 [  5 994]]





**Сравнение метрик**:

- **rubert_tiny2**:
  - Accuracy (точность): 0.96
  - Precision: 0.94 (класс 0), 0.98 (класс 1)
  - Recall: 0.98 (класс 0), 0.95 (класс 1)
  - F1-Score: 0.96 для обоих классов

- **rubert_tiny2_p**:
  - Accuracy (точность): 0.95
  - Precision: 1.00 (класс 0), 0.93 (класс 1)
  - Recall: 0.91 (класс 0), 1.00 (класс 1)
  - F1-Score: 0.95 (класс 0), 0.96 (класс 1)

---

**Анализ путаницы**:

- **rubert_tiny2**:
  - Ошибок для класса 0: 16 (ложноотрицательные)
  - Ошибок для класса 1: 60 (ложноположительные)  

- **rubert_tiny2_p**:
  - Ошибок для класса 0: 76 (ложноотрицательные)
  - Ошибок для класса 1: 5 (ложноположительные)

---

**Вывод**:
- **rubert_tiny2** выглядит более универсальной, так как показывает более стабильные метрики на обоих классах.
- Однако, **rubert_tiny2_p**, минимизирует вероятность ошибки для этого спам сообщений.

#### Строки с FN и FP для модели finetuned_rubert_tiny2

In [51]:
fn_rows = df_test[(df_test["label"] == 1) & (df_test["rubert_tiny2"] == 0)]
fp_rows = df_test[(df_test["label"] == 0) & (df_test["rubert_tiny2"] == 1)]

In [52]:
fn_rows.info()

<class 'pandas.core.frame.DataFrame'>
Index: 60 entries, 1 to 996
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   text               60 non-null     object
 1   label              60 non-null     int64 
 2   emojis             60 non-null     int64 
 3   newlines           60 non-null     int64 
 4   whitespaces        60 non-null     int64 
 5   links              60 non-null     int64 
 6   tags               60 non-null     int64 
 7   text_preprocessed  60 non-null     object
 8   rubert_tiny2       60 non-null     int64 
 9   rubert_tiny2_p     60 non-null     int64 
dtypes: int64(8), object(2)
memory usage: 5.2+ KB


In [53]:
fn_rows.head(500)

,text,label,emojis,newlines,whitespaces,links,tags,text_preprocessed,rubert_tiny2,rubert_tiny2_p
1,دانیالللل,1,0,0,0,0,0,دانیالللل,0,1
8,اره v2ray,1,0,0,0,0,0,اره v2ray,0,1
33,معايا 2022,1,0,0,0,0,0,معايا 2022,0,1
60,هممونو,1,0,0,0,0,0,هممونو,0,1
72,B1 به بالا,1,0,0,0,0,0,b1 به بالا,0,1
78,Saba بیا,1,0,0,0,0,0,saba بیا,0,1
85,13k دهنت,1,0,0,0,0,0,13k دهنت,0,1
128,المفروض 1.2,1,0,0,0,0,0,المفروض 12,0,1
183,23 بهمن,1,0,0,0,0,0,23 بهمن,0,1
185,جايب 0.01 منين,1,0,0,0,0,0,جايب 001 منين,0,1


In [54]:
fp_rows.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16 entries, 4 to 846
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   text               16 non-null     object
 1   label              16 non-null     int64 
 2   emojis             16 non-null     int64 
 3   newlines           16 non-null     int64 
 4   whitespaces        16 non-null     int64 
 5   links              16 non-null     int64 
 6   tags               16 non-null     int64 
 7   text_preprocessed  16 non-null     object
 8   rubert_tiny2       16 non-null     int64 
 9   rubert_tiny2_p     16 non-null     int64 
dtypes: int64(8), object(2)
memory usage: 1.4+ KB


In [55]:
fp_rows.head(500)

,text,label,emojis,newlines,whitespaces,links,tags,text_preprocessed,rubert_tiny2,rubert_tiny2_p
4,"Здравствуйте, напишите свой вопрос в данный ча...",0,0,2,1,1,0,здравствуйте напишите свой вопрос в данный чат...,1,1
151,Здорово! Отличная возможность для молодых пред...,0,0,0,0,0,0,здорово отличная возможность для молодых предп...,1,1
153,Отличная возможность отдохнуть на Черноморском...,0,0,0,0,0,0,отличная возможность отдохнуть на черноморском...,1,1
196,Это отличная возможность вернуться к хакатонам...,0,2,0,0,0,0,это отличная возможность вернуться к хакатонам...,1,1
314,"Ищу кто на 38.03.02\nПишите в личку, будем общ...",0,1,1,0,0,0,ищу кто на 380302 пишите в личку будем общатьс...,1,1
444,Если вопросы какие-нибудь есть - пишите) По во...,0,1,0,0,0,0,если вопросы какиенибудь есть пишите по возмож...,1,1
501,"Отличный отдых, полный активного времяпрепрово...",0,0,0,0,0,0,отличный отдых полный активного времяпрепровож...,1,0
566,"Отлично проведенное мероприятие, полное весель...",0,0,0,0,0,0,отлично проведенное мероприятие полное веселья...,1,1
611,Круто! Цифровой рубль - новинка с потенциалом! 💰,0,1,0,0,0,0,круто цифровой рубль новинка с потенциалом,1,1
613,Зачем нужен цифровой рубль (ЦР):\n1. Платить Ц...,0,0,3,1,0,0,зачем нужен цифровой рубль цр 1 платить цр деш...,1,1


In [61]:
fp_rows[['text', 'label']].to_dict('records')

[{'text': 'Здравствуйте, напишите свой вопрос в данный чат \n\nhttps://t.me/stankinpkchat',
  'label': 0},
 {'text': 'Здорово! Отличная возможность для молодых предпринимателей реализовать свои мечты и найти партнеров. Соревнование в двух лигах, так что каждый найдет свое место. Не упусти шанс и отправляй заявку прямо сейчас!',
  'label': 0},
 {'text': 'Отличная возможность отдохнуть на Черноморском побережье! Пожелаем ребятам высоких результатов в соревнованиях и море положительных эмоций на отдыхе!',
  'label': 0},
 {'text': 'Это отличная возможность вернуться к хакатонам и открыться новым приключениям с командой единомышленников! Не теряйте времени и подавайте заявку на участие прямо сейчас. Удачи в создании компьютерной игры и получении ценных призов!🎮🔥',
  'label': 0},
 {'text': 'Ищу кто на 38.03.02\nПишите в личку, будем общаться, знакомится! ☺️',
  'label': 0},
 {'text': 'Если вопросы какие-нибудь есть - пишите) По возможности постараюсь ответить🙃',
  'label': 0},
 {'text': 'Отл